In [1]:
import numpy as np
import matplotlib.pyplot as plt
from FFT_Computation_by_blocks_utils import *

%matplotlib widget

# Creating the signal and kernel

In [2]:
x, kernel = create_signal_and_kernel(num_samples=3000, mu=0, sig=1, kernel_size=100, kernel_sig=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Traditional filtering (Convolution)

In [3]:
x_filtered = convolution_filter(x, kernel)

# DFT filtering

In [4]:
visualize_zero_padding(x, kernel)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
x_filtered_FT = DFT_filtering(x, kernel)
visualize_DFT_filtering(x_filtered_FT, x_filtered)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# DFT filtering by blocks

In [6]:
number_of_blocks = 3

## Method 1: Overlap add
<img src="images/slide_11-5.png" alt="slide 11-5" width="800">

In [7]:
x_filtered_blocks, x_filtered_overlap_add = overlap_add(x, kernel, n=number_of_blocks)
visualize_overlap_add(x_filtered_blocks, x_filtered_overlap_add, x_filtered, kernel.shape[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Method 2: Overlap save
<img src="images/slide_11-6.png" alt="slide 11-6" width="900">

In [8]:
x_filtered_blocks, x_filtered_overlap_save = overlap_save(x, kernel, number_of_blocks)   
visualize_overlap_save(x_filtered_blocks, x_filtered_overlap_save, x_filtered, kernel.shape[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Error and execution time

In [9]:
visualize_time_and_error(x, kernel, runs=100, ns=[1, 2, 3, 4, 5, 6, 8, 10])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …